In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [49]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import LDA


In [50]:
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [51]:
def From_pyparquet_to_pandascsv2(parquet_path, csv_name) :
    df = pq.read_table(parquet_path).to_pandas()
    csv_save_path = "/Users/youssouf/Documents/simboxv2/" + csv_name + ".csv"
    df.to_csv(csv_save_path, index = False, sep = ";" )
    return os.path.isfile(csv_save_path)

In [52]:
def From_pyparquet_to_pandascsv3(parquet_path) :
    df = pq.read_table(parquet_path).to_pandas()
    csv_save_path = parquet_path.split(".")[0] + ".csv"
    df.to_csv(csv_save_path, index = False, sep = ";" )
    return os.path.isfile(csv_save_path)

In [57]:
def non_detected_and_detected(path_detected ,path_trafic_jour , date) :
    from datetime import datetime

    df_detected = spark.read.option("header", "true").csv(path_detected)
    df_jour = spark.read.option("header", "true").option("delimiter", ";").csv(path_trafic_jour)
      
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Documents/simboxv2/outils_fra_detection.csv")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("date_detection") == date)
        
    df_non_detected = df_fraudeur.join(df_detected , ["msisdn"], "leftanti").select("msisdn") \
                                  .join(df_jour, ["msisdn"] , "left")  
    
    df_detected = df_detected.join(df_jour, ["msisdn"] , "left")
    
    d = datetime.strptime(date , "%d/%m/%Y")
    dt = datetime.strftime(d , '%Y%m%d')

    save_detected_path = "/Users/youssouf/Documents/simboxv2/df_detected_" + dt + ".parquet" 
    df_detected.write.format("parquet").mode("overwrite").save(save_detected_path)
    From_pyparquet_to_pandascsv3(save_detected_path)
    
    save_non_detected_path = "/Users/youssouf/Documents/simboxv2/df_non_detected_" + dt + ".parquet" 
    df_non_detected.write.format("parquet").mode("overwrite").save(save_non_detected_path)
    From_pyparquet_to_pandascsv3(save_non_detected_path)
    
    

In [58]:
non_detected_and_detected("/Users/youssouf/Downloads/prediction_final_kmeans20211007.csv",
                         "/Users/youssouf/Downloads/trafic_global_20211007.csv",
                         "07/10/2021")

Py4JJavaError: An error occurred while calling o310.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.

In [13]:
df_non_detected07 = non_detected("/Users/youssouf/Downloads/prediction_final_kmeans20211007.csv", "07/10/2021")

In [14]:
df_non_detected07.show()

+-------------+
|       msisdn|
+-------------+
|2250700143270|
|2250700196611|
|2250700248515|
|2250700308607|
|2250700308632|
|2250700323676|
|2250700345628|
|2250700473501|
|2250700473524|
|2250700474637|
|2250700474690|
|2250700474711|
|2250700475099|
|2250700475134|
|2250700475135|
|2250700475295|
|2250700557273|
|2250700606402|
|2250700606875|
|2250700689995|
+-------------+
only showing top 20 rows



In [15]:
df_non_detected07.count()

264

In [16]:
df_jour = spark.read.option("delimiter", ";").option("header", "true").csv("/Users/youssouf/Downloads/trafic_global_20211007.csv")
df_jour.show()


+----------+-------------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+
|date_appel|       msisdn|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|
+----------+-------------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+
|  20211007|2250700509019|         3.0|              2.0|          0.0|    0.0|               0.0|          0.0|       17.0|             4.0|         0.0|   0.0|              0.0|             0|                  0|

In [28]:
df_non_detected07_ = df_non_detected07.join(df_jour, ["msisdn"] , "left")
df_non_detected07_.printSchema()
df_non_detected07_.show()

root
 |-- msisdn: string (nullable = true)
 |-- date_appel: string (nullable = true)
 |-- nbre_sms_out: string (nullable = true)
 |-- nbre_dist_sms_out: string (nullable = true)
 |-- nbre_voix_out: string (nullable = true)
 |-- dur_out: string (nullable = true)
 |-- nbre_dist_voix_out: string (nullable = true)
 |-- nbre_cell_out: string (nullable = true)
 |-- nbre_sms_in: string (nullable = true)
 |-- nbre_dist_sms_in: string (nullable = true)
 |-- nbre_voix_in: string (nullable = true)
 |-- dur_in: string (nullable = true)
 |-- nbre_dist_voix_in: string (nullable = true)
 |-- nbre_sms_night: string (nullable = true)
 |-- nbre_dist_sms_night: string (nullable = true)
 |-- nbre_voix_night: string (nullable = true)
 |-- dur_night: string (nullable = true)
 |-- nbre_dist_voix_night: string (nullable = true)

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+----

In [33]:
df_non_detected07_.show(5)

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|
+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+
|2250789658495|      null|        null|             null|         null|   null|              null|         null|       null|            null|        null|  null|             null|          null|               null|

In [32]:
cols = ['nbre_sms_out', 'nbre_dist_sms_out', 'nbre_voix_out', 'dur_out', 'nbre_dist_voix_out', 'nbre_cell_out',
 'nbre_sms_in', 'nbre_dist_sms_in', 'nbre_voix_in', 'dur_in', 'nbre_dist_voix_in', 'nbre_sms_night','nbre_dist_sms_night',
 'nbre_voix_night','dur_night','nbre_dist_voix_night']

for cl in cols :
    df_non_detected07_ = df_non_detected07_.withColumn(col , col(cl).cast(DoubleType()))
    

TypeError: 'str' object is not callable

In [ ]:
df_non_detected07_.describe().show()

In [ ]:
df_non_detected07_.select("date_appel", "nbre_sms_out", "nbre_dist_sms_out", "nbre_voix_out", "dur_out").describe().show()




In [ ]:
df_non_detected07_.select("date_appel", "nbre_dist_voix_out", "nbre_cell_out").describe().show()


In [34]:
df_non_detected07_.write.format("parquet").mode("overwrite").parquet("/Users/youssouf/Documents/simboxv2/df_non_detected07.parquet")



In [37]:
#From_pyparquet_to_pandascsv2("/Users/youssouf/Documents/simboxv2/df_non_detected07.parquet", "df_non_detected07")

In [174]:
df_non_detected07_.select("msisdn").show()

Py4JJavaError: An error occurred while calling o323.showString.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:205)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoin

In [ ]:
df_non_detected07_.select("msisdn").withColumn("m", substring(col("msisdn"), 4, 4)) .show()

In [ ]:
substring()

## KNN APPROACH

In [171]:
df_trans = pd.read_csv("/Users/youssouf/Documents/simboxv2/transformed_yeojonhson.csv", sep = ";")
df_trans.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/youssouf/Documents/simboxv2/transformed_var.csv'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3) # 70% training and 30% test


In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

In [6]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [ ]:
  ['esnode1', 'esnode2'],
        # sniff before doing anything
        sniff_on_start=True,
        # refresh nodes after a node fails to respond
        sniff_on_connection_fail=True,
        # and also every 60 seconds
        sniffer_timeout=60

In [7]:
def delete_by_ids(index, ids):
    query = {
  "query": {
         "bool": {
             "filter":[
                    {
                      "term":
                     {
                        "Age.keyword": ids
                     }
                    }
             ]
         }
  }
}
    res = es.delete_by_query(index=index, body=query)
    pprint(res)

# Pass index and list of id that you want to delete.
delete_by_ids('titanic', "28")

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [ ]:
### Red